In [51]:
from statsmodels.miscmodels.ordinal_model import OrderedModel
import pandas as pd
from pandas.api.types import CategoricalDtype
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

In [52]:
path = '/Users/alexandra/Desktop/ML research/'

In [120]:
y = pd.read_excel(path + 'features.xlsx', usecols=['survey_reliance'])
x = pd.read_excel(path + 'features.xlsx', 
                  usecols=[
                           'total_duration',
                           'percent_gaze_duration_on_ai',
                           #'total_duration_on_ai', *
                            
                           #check if those outputs could be used
                           'Trust',
                           'Familiarity',
                           'Age',
                           'Education',
                           'Gender',
                                                      
                           
                           'percent_duration_task',
                           #'total_duration_on_task',
                           'percent_duration_user', 
                           #'total_duration_on_user', 
                           'first_gaze_on_ai_start_time', 
                           'first_gaze_on_ai_duration',
                           'count_ai2task', 
                           'percent_shift_ai2task', 
                           'count_ai2user',
                           'percent_shift_ai2user',                       
                           'count_task2user', 
                           'percent_shift_task2user',
                           'count_task2ai', 
                           'count_2task',
                           'percent_shift_task2ai',
                           'count_user2ai',
                           'percent_shift_2ai',                            
                           #'percent_shift_user2ai',
                           'count_user2task', 
                           #'percent_shift_user2task'*
                           #'percent_shift_2user', 
                           #'count_2user',
                           #'start_user_ai_acc',
                           #'Orientation'   
                           ])

#x['Orientation'] = x['Orientation'].replace([1],[5.0])


In [121]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.11, shuffle = False)

y_train = y_train.replace([1, 2, 3, 4, 5, 6, 7], ['A', 'B', 'C', 'D', 'E', 'F', 'G'])
cat_type = CategoricalDtype(categories=['A', 'B', 'C', 'D', 'E', 'F', 'G'], ordered=True)
y_train = y_train.astype(cat_type)

#y_test = y_test.replace([1, 2, 3, 4, 5, 6, 7], ['A', 'B', 'C', 'D', 'E', 'F', 'G'])
#cat_type = CategoricalDtype(categories=['A', 'B', 'C', 'D', 'E', 'F', 'G'], ordered=True)
#y_test = y_test.astype(cat_type)

y = y.replace([1, 2, 3, 4, 5, 6, 7], ['A', 'B', 'C', 'D', 'E', 'F', 'G'])
cat_type = CategoricalDtype(categories=['A', 'B', 'C', 'D', 'E', 'F', 'G'], ordered=True)
y = y.astype(cat_type)

In [122]:
mod_prob = OrderedModel(y['survey_reliance'], x, distr='probit')
res_prob = mod_prob.fit(method='bfgs')
res_prob.summary()

Optimization terminated successfully.
         Current function value: 1.639220
         Iterations: 143
         Function evaluations: 147
         Gradient evaluations: 147


/Users/alexandra/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:592: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '


<class 'statsmodels.iolib.summary.Summary'>
"""
                             OrderedModel Results                             
==============================================================================
Dep. Variable:        survey_reliance   Log-Likelihood:                -560.61
Model:                   OrderedModel   AIC:                             1181.
Method:            Maximum Likelihood   BIC:                             1296.
Date:                Wed, 18 Jan 2023                                         
Time:                        03:21:17                                         
No. Observations:                 342                                         
Df Residuals:                     312                                         
Df Model:                          30                                         
===============================================================================================
                                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
total_duration                  0.0017        nan        nan        nan         nan         nan
percent_gaze_duration_on_ai    -0.7479        nan        nan        nan         nan         nan
Age                            -0.0033        nan        nan        nan         nan         nan
Gender                          0.1064        nan        nan        nan         nan         nan
Education                      -0.0085        nan        nan        nan         nan         nan
Familiarity                    -0.1611        nan        nan        nan         nan         nan
Trust                          -0.1245        nan        nan        nan         nan         nan
percent_duration_task           1.2020        nan        nan        nan         nan         nan
percent_duration_user           3.9665        nan        nan        nan         nan         nan
first_gaze_on_ai_start_time     0.0015        nan        nan        nan         nan         nan
first_gaze_on_ai_duration      -0.1895        nan        nan        nan         nan         nan
count_ai2task                   0.1564        nan        nan        nan         nan         nan
percent_shift_ai2task          -1.6150        nan        nan        nan         nan         nan
count_ai2user                   0.3190        nan        nan        nan         nan         nan
percent_shift_ai2user          -2.2938        nan        nan        nan         nan         nan
count_task2user                 0.1664        nan        nan        nan         nan         nan
percent_shift_task2user        -5.4242        nan        nan        nan         nan         nan
count_task2ai                  -0.1627        nan        nan        nan         nan         nan
percent_shift_task2ai          -4.7498        nan        nan        nan         nan         nan
count_user2ai                  -0.2985        nan        nan        nan         nan         nan
count_user2task                -0.1629        nan        nan        nan         nan         nan
percent_shift_2ai               4.1547        nan        nan        nan         nan         nan
percent_shift_2user            -7.7180        nan        nan        nan         nan         nan
count_2task                    -0.0065        nan        nan        nan         nan         nan
A/B                            -5.4976        nan        nan        nan         nan         nan
B/C                             0.0638        nan        nan        nan         nan         nan
C/D                            -1.2284        nan        nan        nan         nan         nan
D/E                            -2.2668        nan        nan        nan         nan         nan
E/F                            -0.5544        nan        nan        nan         nan         nan
F/G                            -0.2867        nan        nan        nan         nan       

In [64]:
y_hat = res_prob.predict(x_test)
value = y_hat.to_numpy()

def expected_value(values, weights):
    values = np.asarray(values)
    weights = np.asarray(weights)
    return (values * weights).sum() 


EV = []
weights = [1,2,3,4,5,6,7]

for i in range(0,38,1):
    values = value[i]
    ev = expected_value(values,weights);
    EV.append(ev)


r_square = mean_squared_error(y_test, EV)
r_square

2.0454635299888166